In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from dotenv import load_dotenv
import nest_asyncio
import os

os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [4]:
import art
from art.multidevice import MultiDeviceBackend

from instances import as_instances_iter, get_filtered_swe_smith_instances_df
from rollout import ModelConfig, rollout

instances = (get_filtered_swe_smith_instances_df().sample(fraction=1.0, shuffle=True, seed=42).head(1).pipe(as_instances_iter))

backend = MultiDeviceBackend()
model = art.TrainableModel(
    name="001",
    project="swebench",
    base_model="Qwen/Qwen2.5-14B-Instruct",
    config=ModelConfig()
)
await model.register(backend)

trajectory_groups = await art.gather_trajectory_groups(
    (
        art.TrajectoryGroup(rollout(model, instance, run_in_thread=False) for _ in range(1)) for instance in instances
    ),
    pbar_total_completion_tokens=False
)

INFO 05-29 00:26:21 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-29 00:26:21 [__init__.py:239] Automatically detected platform cuda.


/home/sky/sky_workdir/src/art/__init__.py:11: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-29 00:26:28 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-29 00:26:28 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA H200. Num GPUs = 2. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-14b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 139.72 GB.
Unsloth: Using conservativeness = 1.0. C

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:00,  4.32it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.65it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.45it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.59it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:00,  4.48it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.81it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.49it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.65it/s]



INFO 05-29 00:26:43 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-29 00:26:43 [model_runner.py:1140] Model loading took 10.6668 GiB and 4.091875 seconds
INFO 05-29 00:26:47 [worker.py:287] Memory profiling takes 3.06 seconds
INFO 05-29 00:26:47 [worker.py:287] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.79) = 109.90GiB
INFO 05-29 00:26:47 [worker.py:287] model weights take 10.67GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 4.26GiB; the rest of the memory reserved for KV Cache is 94.82GiB.
INFO 05-29 00:26:47 [executor_base.py:112] # cuda blocks: 32366, # CPU blocks: 2048
INFO 05-29 00:26:47 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 15.80x
INFO 05-29 00:26:49 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the C

Capturing CUDA graph shapes: 100%|█████████████| 53/53 [00:41<00:00,  1.28it/s]


INFO 05-29 00:27:31 [model_runner.py:1592] Graph capturing finished in 42 secs, took 2.09 GiB
INFO 05-29 00:27:31 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 47.39 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'k_norm', 'pre_feedforward_layernorm', 'q_norm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'k_norm', 'pre_feedforward_layernorm', 'q_norm']


Unsloth 2025.5.1 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


gather:   0%|          | 0/1 [00:00<?, ?it/s]

CancelledError: 

In [7]:
instances = list(get_filtered_swe_smith_instances_df().sample(fraction=1.0, shuffle=True, seed=42).head(1).pipe(as_instances_iter))

In [9]:
await rollout(model, instances[0], run_in_thread=False)

RemoteError: Image build for im-Hb8Ov34oe2IXc7zXQTSDBz failed with the exception:
